In [ ]:
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 

# Ejecutar antes las celdas de `01_access_log_parsing` o elige una ubicación donde exista un archivo de access logs (Apache Combined Log Format) 
log_file_path = '../data/target/access_log_master.log'

## Load Access Log File 

In [ ]:
# Cambia a None para cargar todos los datos, o a un número para establecer un límite
n_samples = None 

with open(log_file_path, 'r') as f:
  if n_samples is not None: 
    sample_lines = []
    for _ in range(n_samples):
      line = f.readline()
      
      # si se acaba el archivo antes de alcanzar el número de muestras definidas
      if not line:    
        break 
      sample_lines.append(line.strip())
  else: 
    # Cargar todas las líneas
    sample_lines = [line.strip() for line in f]

print(f"Sample log lines (Total: {len(sample_lines)}):")
for i,line in enumerate(sample_lines[:10], 1):
  print(f"{i:>{len(str(n_samples))}}) {line}") 

## Load and Parsing a Access Log File using `lars`

In [ ]:
import re 
import warnings
from lars.apache import ApacheSource, COMBINED, ApacheWarning

parsed_logs = []
conflicting_logs = {}
problematic_logs = {}

# Leer todas las lineas primero
with open(log_file_path, 'r') as f:
  all_lines = f.readlines()

# usar catch-warning para capturar advertencias
with warnings.catch_warnings(record=True) as w:
  # convertir todas las advertencias en excepciones o capturarlas
  warnings.simplefilter("always", ApacheWarning)

  with open(log_file_path) as f:
    with ApacheSource(f, log_format=COMBINED) as source:
      for i,row in enumerate(source, 1):
        try:
          record = {
            "remote_host" : row.remote_host,
            "ident" : row.ident,
            "remote_user" : row.remote_user, 
            "time" : row.time,
            "request_method" : row.request.method,
            "request_url_scheme" : row.request.url.scheme, 
            "request_url_netloc" : row.request.url.netloc,
            "request_url_path_str" : row.request.url.path_str,
            "request_url_params" : row.request.url.params, 
            "request_url_query_str" : row.request.url.query_str,
            "request_url_fragment" : row.request.url.fragment,
            "request_protocol": row.request.protocol,
            "status" : row.status,
            "size" : row.size,
            "req_Referer" : row.req_Referer,
            "req_User_agent" : row.req_User_agent
          }
          parsed_logs.append(record)
        except Exception as e:
          #print(f"Error en fila {i}: {e}")
          conflicting_logs[i] = {
            'error': str(e), 
            'line_content': all_lines[i-1].strip() if i <= len(all_lines) else "no disponible"
          }
  
  # procesar las advertencias capturadas
  for warning in w:
    if issubclass(warning.category, ApacheWarning):
      msg = str(warning.message)
      match = re.search(r'Line (\d+):', msg)
      if match:
        line_num = int(match.group(1))
        if line_num <= len(all_lines):
            problematic_logs[line_num] = {
              'line_content': all_lines[line_num - 1].strip(),
              'warning_message': msg, 
              'category': warning.category.__name__,
            }

Mientras que `conflicting_logs` captura logs con errores menores que impiden procesar completamente una línea. Estos fallos en la extracción se debe a errores en la consulta (`request.url`) que puede deberse a logs malignos. En cambio, `problematic_logs` captura logs con problemas mayores, lo cual es motivo para que no se parseen los logs (no se encuentran en `parsed_logs`). Los logs se pueden considerar en una primera instancia como anomalías debido a que no siguen la estructura común de los Apache access logs de formato combinado

In [ ]:
for warning in list(problematic_logs.items())[:10]:
  idx = warning[0]
  line_content = warning[1]['line_content']
  print(f"{idx}) {line_content}")

Dentro de los logs problemáticos se notan se forma rápida:
- No presentan un método HTTP válido (como GET, POST, etc) o simplemente NO presentan uno
- Presentan un comportamiento sospechoso como: No hay presencia de Referer ni User-Agent 

In [ ]:
from parse import parse 

pattern = '{ip_client} {ident} {auth_user} [{timestamp}] "{method} {request} {protocol}" {status:d} {size:d} "{referrer}" "{agent}"'

parsed_logs = []
for conflict in list(conflicting_logs.items()):
  idx = conflict[0]
  line_content = conflict[1]['line_content']
  try: 
    parsed = parse(pattern, line_content.strip())
    parsed_logs.append(parsed.named)
  except Exception as e:
    print(f"Line {idx}: {line_content}")
    print(f"Error: {e}")

A pesar de que con `lars` los logs pertenecientes a `conflicting logs` no se logran parsear correctamente, se pudo hacer ese proceso con `parse`. La diferencia principal entre ambas es en la forma de extraer las características de la petición (`request` en el patrón diseñado para `parse` y `request.url` para `lars`) 

In [ ]:
total_problematic_logs =  len(problematic_logs.keys())
total_conflicting_logs =  len(conflicting_logs.keys())
total_logs = len(parsed_logs) + total_problematic_logs + total_conflicting_logs 

print(f"Total: {total_logs}")
print(f"Cantidad de Problematic Logs: {total_problematic_logs} ({(total_problematic_logs / total)*100 if total != 0 else 0}%)")
print(f"Cantidad de Conflicting Logs: {total_conflicting_logs} ({(total_conflicting_logs / total)*100 if total != 0 else 0}%)")
print(f"Cantidad de Parsed Logs: {len(parsed_logs)} ({(len(parsed_logs) / total)*100 if total != 0 else 0}%)")

### Análisis de Logs Problemáticos

Estos logs deben ser analizados porque están fuera de lo común y necesitan otra forma de parsear estos y del resultado del parseo entonces analizar características como `status`, `ip-client` y `user-agent` 

In [ ]:
max_idx = max(problematic_logs.keys())

logs = list(problematic_logs.items()) 

for log in logs:
  idx = log[0]
  line_content = log[1]['line_content']
  print(f"{idx:>{len(str(max_idx))}}) {line_content}") 

Presentan:
- Cadenas bytes en formato hexademical en las solicitudes HTTP (tanto en método, protocolo y url)
- No existe Referer, ni User-Agent
- Las solicitudes HTTP solo poseen solo método (precisamente GET) 
- Las pocas solicitudes HTTP (las que no están formadas por bytes en formato hexademical) no presentan método y tienen cadenas muy extrañas, tienen direcciones IP o tienen mensajes JSON-RPC (protocolos de minería de criptomonedas)

In [ ]:
from parse import parse 

pattern = '{ip_client} {ident} {auth_user} [{timestamp}] {request_http} {status:d} {size:d} "{referrer}" "{agent}"'

parsed_logs = []
for problematic in list(problematic_logs.items()):
  idx = problematic[0]
  line_content = problematic[1]['line_content']
  try: 
    parsed = parse(pattern, line_content.strip())
    parsed_logs.append(parsed.named)
  except Exception as e:
    print(f"Line {idx}: {line_content}")
    print(f"Error: {e}")

df = pd.DataFrame(parsed_logs)
status_distribution = df['status'].value_counts().sort_index()
display(status_distribution)

**Análisis de Distribución de Código de Estado HTTP**:
- `301`: El recurso solicitado tiene una nueva URL permanente 
- `400`: El servidor no puede entender la solicitud debido a una sintaxis inválida 
- `408`: El servidor esperó demasiado tiempo a que el cliente enviara la solicitud completa y cerró la conexión inactiva
- `499`: El cliente cerro la solicitud antes de que el servidor pudiera responder

Para códigos 400, se va revisar la dirección IP de origen y el agente de usuario $\to$ Un alto volumen de errores 400 provenientes de unas pocas IPs o de agentes de usuario inusuales (como herramientas de escaneo) puede indicar actividad maliciosa automatizada. 

Para códigos 408 y 499: Estos códigos suelen ser síntomas de que algo en el servidor no funciona de manera óptima $\to$ Un aumento repentino suele estar relacionado con una alta carga en el servidor, tiempos de respuesta lentos en la aplicación o problemas de red

In [ ]:
# Filtrar logs con los códigos de estado de interés
status_codes_to_check = [400, 408, 499]
filtered_logs = df[df['status'].isin(status_codes_to_check)]

# Agrupar por IP para identificar posibles orígenes problemáticos
ip_analysis = filtered_logs.groupby(['status', 'ip_client']).size().reset_index(name='count')
print(ip_analysis.sort_values(by='count', ascending=False).head(20))  # Ver las 20 IPs más activas

# Agrupar por agente de usuario
agent_analysis = filtered_logs.groupby(['status', 'agent']).size().reset_index(name='count')
print(agent_analysis.sort_values(by='count', ascending=False).head(10))  # Ver los 10 agentes más comunes

Estos resultados muestran una **actividad maliciosa**: patrón de cientos de errores 400, concentrados en pocas IPs, cada una con rangos muy diferentes, y con agente vacío, no es un comportamiento normal de un usuario o buscador. 

## Análisis Exploratorio de Datos sobre Access Log Master File

Debido a que los logs conflictivos se pudieron parsear correctamente con el siguiente patrón y `parse`.  

`'{ip_client} {ident} {auth_user} [{timestamp}] "{method} {request} {protocol}" {status:d} {size:d} "{referrer}" "{agent}"'`

Entonces se va a parsear todos los access log con este patrón desde el archivo inicial y se va a guardar los resultados en un dataframe y guardar este en un archivo CSV. Luego se va a preprocesar la información que proporciona este archivo de logs.

In [ ]:
# Eliminar los logs que se conocen que son maliciosos 
# Parsear primero todos los access log (ya están cargados en una de las primeras celdas), se tiene que solamente introducir el patrón y parsear todos los logs
# Agregar una característica adicional (method+request+protocol) porque con esta característica se puede guardar la petición HTTP de las solicitudes maliciosas
# Finalmente se parsean los logs maliciosos y se agregan al dataset

### TODO

- Análisis Exploratorio de Datos sobre los DataFrames
  - Distribución de Tiempo y Consulta (saber la densidad de consulta por día)
  - Distribución de `method`, `protocol`, `status`, `size`, `referer`, `user_agent`
  - 